In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Wed Jul  8 13:31:41 PDT 2020


In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/ethiopia/2020_06_28_12_40_56/count_data_incomplete/',
               '/share/costeffectiveness/results/vivarium_conic_lsff/india/2020_06_26_20_35_00/count_data/',
              '/share/costeffectiveness/results/vivarium_conic_lsff/nigeria/2020_06_26_20_28_27/count_data/']

locations = ['Ethiopia','India','Nigeria']

In [3]:
eth = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
eth.head()

# lacks stratification by year... this was a countspace transformation issue 
# (Kjell fixed specifically for India and Nigeria, but looks like fix didn't happen for Ethiopia)

,year,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,covered,covered,person_time,29,baseline,0.0
1,2020,1_to_4,covered,covered,person_time,29,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,covered,covered,person_time,29,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,covered,covered,person_time,29,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,covered,covered,person_time,223,baseline,0.0


In [11]:
ind = pd.read_hdf(output_dirs[2] + 'person_time.hdf')
ind.head()

,year,age_group,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,covered,covered,person_time,21,baseline,0.0
1,2020,1_to_4,covered,covered,person_time,21,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,covered,covered,person_time,21,iron_folic_acid_fortification_scale_up,0.0
3,2020,1_to_4,covered,covered,person_time,21,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,covered,covered,person_time,29,baseline,0.0


# Vitamin A Fortification Coverage

In [4]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR
master_cov = pd.DataFrame()
for i in range(len(output_dirs)):
    data = pd.read_hdf(output_dirs[i] + 'state_person_time.hdf')
    data = data.loc[data.scenario.isin(['baseline','vitamin_a_fortification_scale_up'])] # look only at baseline and vitamin a scenario
    data = data.loc[data.cause.str.contains('diarrheal')]
    data['location'] = locations[i]
    covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
    covered = covered.groupby(['location','scenario','year','input_draw']).sum() # sum over age group, sex
    overall = data.groupby(['location','scenario','year','input_draw']).sum()
    coverage_proportion = covered / overall
    master_cov = master_cov.append(coverage_proportion)
master_cov = master_cov.reset_index().groupby(['location','year','scenario']).describe().drop(columns='input_draw')
master_cov

value                      \
                                               count      mean       std   
location year scenario                                                     
Ethiopia 2020 baseline                          14.0  0.002678  0.004440   
              vitamin_a_fortification_scale_up  14.0  0.002678  0.004440   
         2021 baseline                          14.0  0.002657  0.004400   
              vitamin_a_fortification_scale_up  14.0  0.447769  0.025632   
         2022 baseline                          14.0  0.002672  0.004427   
              vitamin_a_fortification_scale_up  14.0  0.460232  0.025826   
         2023 baseline                          14.0  0.002650  0.004377   
              vitamin_a_fortification_scale_up  14.0  0.469107  0.025963   
India    2020 baseline                          25.0  0.243076  0.006473   
              vitamin_a_fortification_scale_up  25.0  0.243076  0.006473   
         2021 baseline                          25.0  0.243025  0.006396   
              vitamin_a_fortification_scale_up  25.0  0.895753  0.004559   
         2022 baseline                          25.0  0.242912  0.006337   
              vitamin_a_fortification_scale_up  25.0  0.909404  0.004049   
         2023 baseline                          25.0  0.242816  0.006363   
              vitamin_a_fortification_scale_up  25.0  0.918355  0.003817   
Nigeria  2020 baseline                          25.0  0.071725  0.002431   
              vitamin_a_fortification_scale_up  25.0  0.071725  0.002431   
         2021 baseline                          25.0  0.071776  0.002448   
              vitamin_a_fortification_scale_up  25.0  0.282402  0.005964   
         2022 baseline                          25.0  0.071874  0.002410   
              vitamin_a_fortification_scale_up  25.0  0.353734  0.005411   
         2023 baseline                          25.0  0.071888  0.002326   
              vitamin_a_fortification_scale_up  25.0  0.416611  0.004975   

                                                                              \
                                                     min       25%       50%   
location year scenario                                                         
Ethiopia 2020 baseline                          0.000000  0.000000  0.000000   
              vitamin_a_fortification_scale_up  0.000000  0.000000  0.000000   
         2021 baseline                          0.000000  0.000000  0.000000   
              vitamin_a_fortification_scale_up  0.409737  0.430417  0.449821   
         2022 baseline                          0.000000  0.000000  0.000000   
              vitamin_a_fortification_scale_up  0.421585  0.442661  0.463198   
         2023 baseline                          0.000000  0.000000  0.000000   
              vitamin_a_fortification_scale_up  0.429483  0.451747  0.472651   
India    2020 baseline                          0.232244  0.238246  0.244501   
              vitamin_a_fortification_scale_up  0.232244  0.238246  0.244501   
         2021 baseline                          0.232648  0.238031  0.244324   
              vitamin_a_fortification_scale_up  0.887811  0.892917  0.896552   
         2022 baseline                          0.232701  0.237987  0.243959   
              vitamin_a_fortification_scale_up  0.901938  0.907249  0.910242   
         2023 baseline                          0.231933  0.237865  0.243829   
              vitamin_a_fortification_scale_up  0.911560  0.916129  0.919325   
Nigeria  2020 baseline                          0.065993  0.070154  0.071803   
              vitamin_a_fortification_scale_up  0.065993  0.070154  0.071803   
         2021 baseline                          0.065987  0.070364  0.071737   
              vitamin_a_fortification_scale_up  0.268468  0.278510  0.282003   
         2022 baseline                          0.066047  0.070368  0.071708   
              vitamin_a_fortification_scale_up  0.340685  0.350266  0.3535

# Folic Acid Fortification Coverage - Folic Acid Scenario

In [5]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR
master_cov = pd.DataFrame()
for i in range(len(output_dirs)):
    data = pd.read_hdf(output_dirs[i] + 'state_person_time.hdf')
    data = data.loc[data.scenario.isin(['baseline','folic_acid_fortification_scale_up'])] 
    data = data.loc[data.cause.str.contains('diarrheal')]
    data['location'] = locations[i]
    known = data.loc[data.folic_acid_fortification_group != 'unknown']
    covered = known.loc[known.folic_acid_fortification_group == 'covered'] 
    covered = covered.groupby(['location','scenario','year','input_draw']).sum() # sum over age group, sex
    overall = known.groupby(['location','scenario','year','input_draw']).sum()
    coverage_proportion = covered / overall
    master_cov = master_cov.append(coverage_proportion)
master_cov = master_cov.reset_index().groupby(['location','year','scenario']).describe().drop(columns='input_draw')
master_cov

value                      \
                                                count      mean       std   
location year scenario                                                      
Ethiopia 2020 baseline                           14.0  0.000447  0.001192   
              folic_acid_fortification_scale_up  14.0  0.000447  0.001192   
         2021 baseline                           14.0  0.000573  0.001306   
              folic_acid_fortification_scale_up  14.0  0.000573  0.001306   
         2022 baseline                           14.0  0.000572  0.001231   
              folic_acid_fortification_scale_up  14.0  0.024861  0.017312   
         2023 baseline                           14.0  0.000558  0.001202   
              folic_acid_fortification_scale_up  14.0  0.054589  0.036616   
India    2020 baseline                           25.0  0.062878  0.004240   
              folic_acid_fortification_scale_up  25.0  0.062878  0.004240   
         2021 baseline                           25.0  0.063007  0.004278   
              folic_acid_fortification_scale_up  25.0  0.063007  0.004278   
         2022 baseline                           25.0  0.063081  0.004281   
              folic_acid_fortification_scale_up  25.0  0.069885  0.004451   
         2023 baseline                           25.0  0.063095  0.004371   
              folic_acid_fortification_scale_up  25.0  0.091690  0.004863   
Nigeria  2020 baseline                           25.0  0.083313  0.004748   
              folic_acid_fortification_scale_up  25.0  0.083313  0.004748   
         2021 baseline                           25.0  0.083294  0.004718   
              folic_acid_fortification_scale_up  25.0  0.083294  0.004718   
         2022 baseline                           25.0  0.083304  0.004495   
              folic_acid_fortification_scale_up  25.0  0.117490  0.004779   
         2023 baseline                           25.0  0.083382  0.004455   
              folic_acid_fortification_scale_up  25.0  0.156650  0.005144   

                                                                               \
                                                      min       25%       50%   
location year scenario                                                          
Ethiopia 2020 baseline                           0.000000  0.000000  0.000000   
              folic_acid_fortification_scale_up  0.000000  0.000000  0.000000   
         2021 baseline                           0.000000  0.000000  0.000003   
              folic_acid_fortification_scale_up  0.000000  0.000000  0.000003   
         2022 baseline                           0.000000  0.000000  0.000012   
              folic_acid_fortification_scale_up  0.002965  0.008754  0.022861   
         2023 baseline                           0.000000  0.000000  0.000012   
              folic_acid_fortification_scale_up  0.007087  0.020989  0.052556   
India    2020 baseline                           0.055228  0.058447  0.063810   
              folic_acid_fortification_scale_up  0.055228  0.058447  0.063810   
         2021 baseline                           0.055451  0.058693  0.063813   
              folic_acid_fortification_scale_up  0.055451  0.058693  0.063813   
         2022 baseline                           0.055447  0.059062  0.063552   
              folic_acid_fortification_scale_up  0.062010  0.065567  0.070268   
         2023 baseline                           0.055226  0.058716  0.063181   
              folic_acid_fortification_scale_up  0.082934  0.087061  0.091428   
Nigeria  2020 baseline                           0.074437  0.079494  0.083648   
              folic_acid_fortification_scale_up  0.074437  0.079494  0.083648   
         2021 baseline                           0.075180  0.080870  0.083448   
              folic_acid_fortification_scale_up  0.075180  0.080870  0.083448   
         2022 baseline                           0.075120  0.081558  0.083285   
              folic_acid

# Folic Acid Fortification Coverage - Iron Folic Acid Scenario

In [6]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR
master_cov = pd.DataFrame()
for i in range(len(output_dirs)):
    data = pd.read_hdf(output_dirs[i] + 'state_person_time.hdf')
    data = data.loc[data.scenario.isin(['baseline','iron_folic_acid_fortification_scale_up'])] 
    data = data.loc[data.cause.str.contains('diarrheal')]
    data['location'] = locations[i]
    known = data.loc[data.folic_acid_fortification_group != 'unknown']
    covered = known.loc[known.folic_acid_fortification_group == 'covered'] 
    covered = covered.groupby(['location','scenario','year','input_draw']).sum() # sum over age group, sex
    overall = known.groupby(['location','scenario','year','input_draw']).sum()
    coverage_proportion = covered / overall
    master_cov = master_cov.append(coverage_proportion)
master_cov = master_cov.reset_index().groupby(['location','year','scenario']).describe().drop(columns='input_draw')
master_cov

value            \
                                                     count      mean   
location year scenario                                                 
Ethiopia 2020 baseline                                14.0  0.000447   
              iron_folic_acid_fortification_scale_up  14.0  0.000447   
         2021 baseline                                14.0  0.000573   
              iron_folic_acid_fortification_scale_up  14.0  0.000573   
         2022 baseline                                14.0  0.000572   
              iron_folic_acid_fortification_scale_up  14.0  0.024862   
         2023 baseline                                14.0  0.000558   
              iron_folic_acid_fortification_scale_up  14.0  0.054592   
India    2020 baseline                                25.0  0.062878   
              iron_folic_acid_fortification_scale_up  25.0  0.062878   
         2021 baseline                                25.0  0.063007   
              iron_folic_acid_fortification_scale_up  25.0  0.063007   
         2022 baseline                                25.0  0.063081   
              iron_folic_acid_fortification_scale_up  25.0  0.069886   
         2023 baseline                                25.0  0.063095   
              iron_folic_acid_fortification_scale_up  25.0  0.091692   
Nigeria  2020 baseline                                25.0  0.083313   
              iron_folic_acid_fortification_scale_up  25.0  0.083313   
         2021 baseline                                25.0  0.083294   
              iron_folic_acid_fortification_scale_up  25.0  0.083294   
         2022 baseline                                25.0  0.083304   
              iron_folic_acid_fortification_scale_up  25.0  0.117495   
         2023 baseline                                25.0  0.083382   
              iron_folic_acid_fortification_scale_up  25.0  0.156659   

                                                                          \
                                                           std       min   
location year scenario                                                     
Ethiopia 2020 baseline                                0.001192  0.000000   
              iron_folic_acid_fortification_scale_up  0.001192  0.000000   
         2021 baseline                                0.001306  0.000000   
              iron_folic_acid_fortification_scale_up  0.001306  0.000000   
         2022 baseline                                0.001231  0.000000   
              iron_folic_acid_fortification_scale_up  0.017313  0.002965   
         2023 baseline                                0.001202  0.000000   
              iron_folic_acid_fortification_scale_up  0.036618  0.007087   
India    2020 baseline                                0.004240  0.055228   
              iron_folic_acid_fortification_scale_up  0.004240  0.055228   
         2021 baseline                                0.004278  0.055451   
              iron_folic_acid_fortification_scale_up  0.004278  0.055451   
         2022 baseline                                0.004281  0.055447   
              iron_folic_acid_fortification_scale_up  0.004450  0.062011   
         2023 baseline                                0.004371  0.055226   
              iron_folic_acid_fortification_scale_up  0.004862  0.082943   
Nigeria  2020 baseline                                0.004748  0.074437   
              iron_folic_acid_fortification_scale_up  0.004748  0.074437   
         2021 baseline                                0.004718  0.075180   
              iron_folic_acid_fortification_scale_up  0.004718  0.075179   
         2022 baseline                                0.004495  0.075120   
              iron_folic_acid_fortification_scale_up  0.004776  0.108740   
         2023 baseline                                0.004455  0.075404   
              iron_folic_acid_fortification_scale_up  0.005143  0.147270   

                                                       

# Check Ethiopia coverage parameters by DRAW

In [11]:
data = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
data = data.loc[data.scenario.isin(['baseline','vitamin_a_fortification_scale_up'])] # look only at baseline and vitamin a scenario
data = data.loc[data.cause.str.contains('diarrheal')]
data['location'] = locations[0]
covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
covered = covered.groupby(['location','scenario','year','input_draw']).sum() # sum over age group, sex
overall = data.groupby(['location','scenario','year','input_draw']).sum()
coverage_proportion = covered / overall
coverage_proportion = coverage_proportion.reset_index()
baseline = coverage_proportion.loc[coverage_proportion.scenario == 'baseline']
baseline = baseline.set_index(['location','year','input_draw']).drop(columns='scenario')
intervention = coverage_proportion.loc[coverage_proportion.scenario == 'vitamin_a_fortification_scale_up']
intervention = intervention.set_index(['location','year','input_draw']).drop(columns='scenario')
cov_diff = intervention - baseline
cov_diff['value'].min()

0.0

In [19]:
data = pd.read_hdf(output_dirs[0] + 'state_person_time.hdf')
data = data.loc[data.scenario.isin(['baseline','folic_acid_fortification_scale_up'])] # look only at baseline and vitamin a scenario
data = data.loc[data.cause.str.contains('diarrheal')]
data['location'] = locations[0]
known = data.loc[data.folic_acid_fortification_group != 'unknown']
covered = known.loc[known.folic_acid_fortification_group == 'covered'] 
covered = covered.groupby(['location','scenario','year','input_draw']).sum() # sum over age group, sex
overall = known.groupby(['location','scenario','year','input_draw']).sum()
coverage_proportion = covered / overall
coverage_proportion = coverage_proportion.reset_index()
baseline = coverage_proportion.loc[coverage_proportion.scenario == 'baseline']
baseline = baseline.set_index(['location','year','input_draw']).drop(columns='scenario')
intervention = coverage_proportion.loc[coverage_proportion.scenario == 'folic_acid_fortification_scale_up']
intervention = intervention.set_index(['location','year','input_draw']).drop(columns='scenario')
cov_diff = intervention - baseline
cov_diff

value
location year input_draw              
Ethiopia 2020 29          0.000000e+00
              223         0.000000e+00
              232         0.000000e+00
              357         2.710505e-20
              394         0.000000e+00
              514         0.000000e+00
              524        -8.673617e-19
              602         0.000000e+00
              629         0.000000e+00
              650        -6.776264e-21
              680         0.000000e+00
              733         0.000000e+00
              829        -6.776264e-21
              946         0.000000e+00
         2021 29          0.000000e+00
              223         0.000000e+00
              232         0.000000e+00
              357         2.710505e-20
              394         0.000000e+00
              514         0.000000e+00
              524         8.673617e-19
              602         0.000000e+00
              629         0.000000e+00
              650        -6.776264e-21
              680         0.000000e+00
              733         0.000000e+00
              829         0.000000e+00
              946         0.000000e+00
         2022 29          7.917155e-03
              223         9.714674e-03
              232         4.221369e-02
              357         1.983543e-02
              394         7.424286e-03
              514         2.728553e-02
              524         2.892123e-02
              602         2.965247e-03
              629         5.472730e-02
              650         2.568539e-02
              680         1.965190e-02
              733         5.459288e-02
              829         3.067845e-02
              946         8.433739e-03
         2023 29          1.885422e-02
              223         2.314550e-02
              232         9.265410e-02
              357         4.539274e-02
              394         1.785122e-02
              514         5.886756e-02
              524         6.589255e-02
              602         7.086791e-03
              629         1.198063e-01
              650         5.866387e-02
              680         4.641008e-02
              733         1.178668e-01
              829         6.366503e-02
              946         2.026993e-02

In [21]:
cov_diff['value'].min()

-8.673617379884035e-19

In [23]:
scale_up = cov_diff.reset_index()
scale_up = scale_up.loc[scale_up.year.isin(['2022','2023'])]
scale_up['value'].min()

0.002965246865147114

## Conclusion

There is slightly lower coverage in the baseline scenario than the intervention scenario in 2020/2021 (before intervention scale-up), but this is not the case for any draw in 2022/2023 when the scale up should occur. This implies that coverage parameter b is always higher than coverage parameter a at the draw level, as intended. However, this does not confirm that a is always lower than b at the *random seed* level within each draw. In order to confirm this, we would need to analyze the output.hdf file that has these records.